In [1]:
import numpy as np
import pandas as pd
import os

import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

import sklearn, sklearn.linear_model, sklearn.metrics, sklearn.pipeline,sklearn.ensemble

import matplotlib
import scipy.stats

matplotlib.rcParams.update({'font.size': 16})

In [2]:
data_dir = '/lustre/groups/epigenereg01/workspace/projects/vale/MLM/motif_predictions/'

In [3]:
models = { 'Species-agnostic':'species_agnostic/probas.pickle','Species-aware':'species_aware/probas.pickle', 
          'DNABERT': 'dnabert/default/*.pickle', '11-mer':'K-mer/11_mer.pickle','13-mer':'K-mer/13_mer.pickle',
          'PhyloP-100way': '../PhyloP/PhyloP100_3UTR.pickle' ,'PhyloP-241way': '../PhyloP/PhyloP241_3UTR.pickle', 'NT-MS-v2-500M': 'split_75_25/ntrans/NT-MS-v2-500M/*.pickle'}

# Get scores from probabilities

In [4]:
def get_model(glob_path):
    res = {}
    for probas_file in glob(glob_path):
        #print(probas_file)
        with open(probas_file, 'rb') as f:
            model_probas = pickle.load(f)
            if len(model_probas)==2:
                res.update({model_probas[0]:model_probas[1]})
            else:
                res.update(dict(model_probas))
    return res

In [5]:
all_model_probas = {}

for model_name in models:
    all_model_probas[model_name] = get_model(data_dir + models[model_name])
    print(f'{model_name} loaded, {len(all_model_probas[model_name])} sequences')

Species-agnostic loaded, 18134 sequences
Species-aware loaded, 18134 sequences
DNABERT loaded, 18134 sequences
11-mer loaded, 18134 sequences
13-mer loaded, 18134 sequences
PhyloP-100way loaded, 18178 sequences
PhyloP-241way loaded, 18178 sequences
NT-MS-v2-500M loaded, 4245 sequences


In [6]:
def reverse_complement(seq):
    '''
    Take sequence reverse complement
    '''
    compl_dict = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}
    compl_seq = ''.join([compl_dict.get(x,x) for x in seq])
    rev_seq = compl_seq[::-1]
    return rev_seq

In [7]:
mapping = {'A':0,'C':1,'G':2,'T':3}

In [8]:
human_fasta = data_dir + '../fasta/240_species/species/Homo_sapiens.fa' #3'UTR on hegative strand should  be reverse complemented

human_utr = defaultdict(str)

with open(human_fasta, 'r') as f:
    for line in f:
        if line.startswith('>'):
            seq_name = line[1:].split(':')[0]
        else:
            human_utr[seq_name] += line.rstrip().upper()

In [9]:
utr_variants = pd.read_csv(data_dir+'../perbase_pred/variants_snp.tsv', sep='\t') 

In [10]:
# take reverse complement of ref and alt for variants in genes on the negative strand 
# since model predictions are reverse complemented for these sequences
# this is already taken into account for pos_rel (see dataprep)

utr_variants.ref = utr_variants.apply(lambda x:reverse_complement(x.ref) if x.strand=='-' else x.ref, axis=1)
utr_variants.alt = utr_variants.apply(lambda x:reverse_complement(x.alt) if x.strand=='-' else x.alt, axis=1)

In [11]:
# get PhyloP conservation scores at variant positions

for model_name in ('PhyloP-100way','PhyloP-241way',):

    print(model_name)

    probas = all_model_probas[model_name]
    
    for var_idx, var in tqdm(utr_variants.iterrows(), total=len(utr_variants)):
        if var.seq_name in probas.keys() and var.seq_name in human_utr.keys():
            if var.vartype=='SNP':
                assert human_utr[var.seq_name][var.pos_rel] == var.ref
                utr_variants.at[var_idx,model_name+'_ref'] = probas[var.seq_name][var.pos_rel]
            else:
                if var.vartype=='INS':
                    left, right = var.pos_rel-2, var.pos_rel+2
                else:
                    if var.strand=='+':
                        left, right = var.pos_rel, var.pos_rel+len(var.ref)
                    else:
                        left, right = var.pos_rel-len(var.ref), var.pos_rel
                    assert human_utr[var.seq_name][left:right] == var.ref
                utr_variants.at[var_idx,model_name+'_ref'] = np.mean(probas[var.seq_name][left:right])

PhyloP-100way


100%|██████████| 42411/42411 [00:06<00:00, 6895.15it/s]


PhyloP-241way


100%|██████████| 42411/42411 [00:05<00:00, 7937.80it/s] 


In [12]:
def add_model_res(model_name):

    print(model_name)

    probas = all_model_probas[model_name]
        
    for var_idx, var in tqdm(utr_variants.iterrows(), total=len(utr_variants)):
        if var.seq_name in probas.keys() and var.seq_name in human_utr.keys():
            if var.vartype=='SNP':
                assert human_utr[var.seq_name][var.pos_rel] == var.ref
                utr_variants.at[var_idx, model_name+'_alt'] = probas[var.seq_name][var.pos_rel, mapping[var.alt]]
                utr_variants.at[var_idx, model_name+'_ref'] = probas[var.seq_name][var.pos_rel, mapping[var.ref]]
            else:
                if var.vartype=='INS':
                    left, right = var.pos_rel-2, var.pos_rel+2
                else:
                    if var.strand=='+':
                        left, right = var.pos_rel, var.pos_rel+len(var.ref)
                    else:
                        left, right = var.pos_rel-len(var.ref), var.pos_rel
                ref_score = []
                seq = human_utr[var.seq_name]
                assert seq[left:right] == var.ref
                for pos_rel in range(max(left,0),min(right,len(seq))):
                    ref_score.append(probas[var.seq_name][pos_rel, mapping[seq[pos_rel]]]) 
                    #ref_score.append(np.max(probas[var.seq_name][pos_rel])) 
                utr_variants.at[var_idx, model_name+'_ref'] = np.mean(ref_score)

In [13]:
for model_name in ('Species-aware', 'Species-agnostic', 'DNABERT', '11-mer','13-mer', 'NT-MS-v2-500M'):
    add_model_res(model_name)

Species-aware


100%|██████████| 42411/42411 [00:07<00:00, 5808.82it/s]


Species-agnostic


100%|██████████| 42411/42411 [00:07<00:00, 5386.17it/s]


DNABERT


...

100%|██████████| 42411/42411 [00:07<00:00, 5340.82it/s]


13-mer


100%|██████████| 42411/42411 [00:07<00:00, 5455.01it/s]


NT-MS-v2-500M


100%|██████████| 42411/42411 [00:03<00:00, 12388.68it/s]


# Get scores from embeddings

In [132]:
embeddings_dir =  data_dir + '../perbase_pred/embeddings_scores/'

def concat_embeddings(model_name):
    '''
    collect embeddings from all batches and save in a single file
    '''
    print(f'loadding embeddings for {model_name}')
    res = []
    for emb_file in tqdm(glob(embeddings_dir + f'{model_name}/chr*.pickle')):
        with open(emb_file, 'rb') as f:
            seq_names, embeddings, logprobas = pickle.load(f)
            if len(logprobas)==0:
                logprobas = [[] for _ in range(len(seq_names))]            
            res.extend(zip(seq_names,embeddings,logprobas))
    with open(embeddings_dir + f'{model_name}/embeddings.pickle', 'wb') as f:
        print(f'putting all embeddings into a single file')
        pickle.dump(res,f)
        print(f'deleting batch files')
        #for f in glob(embeddings_dir + f'{model_name}/chr*.pickle'):
        #    os.remove(f)
        print('done')

In [140]:
def compute_embeddings_score(seq_names,embeddings,losses,model_name):
    res = []
    loss_ref_avg, loss_ref_central, loss_alt_avg, loss_alt_central = None, None, None, None #we don't compute score base on losses here
    for idx in range(0,len(embeddings),2):
        assert seq_names[idx]==seq_names[idx+1].replace('alt','ref')
        emb_ref, emb_alt = embeddings[idx], embeddings[idx+1]
        l2 = np.linalg.norm(emb_ref-emb_alt)
        l1 = np.linalg.norm((emb_ref-emb_alt), ord=1)
        dot = np.dot(emb_ref,emb_alt)
        cosine = dot/(np.linalg.norm(emb_ref)*np.linalg.norm(emb_alt))
        #loss_ref, loss_alt = losses[idx], losses[idx+1]
        #if len(loss_ref)>0:
        #    loss_ref_avg, loss_ref_central = np.mean(loss_ref), loss_ref[len(loss_ref)//2]
        #    loss_alt_avg, loss_alt_central = np.mean(loss_alt), loss_alt[len(loss_alt)//2]
        #else:
        #    loss_ref_avg, loss_ref_central, loss_alt_avg, loss_alt_central = None, None, None, None
        varname = seq_names[idx].replace('_ref','').split('_')
        res.append((varname[0],int(varname[1]),varname[2],varname[3],l1,l2,dot,cosine,loss_ref_avg, loss_ref_central,loss_alt_avg, loss_alt_central))
        #print(f'{varname[0]}\t{varname[1]}\t{varname[2]}\t{varname[3]}\t{l1}\t{l2}\t{dot}\t{cosine}\t{loss_ref_avg}\t{loss_ref_central}\t{loss_alt_avg}\t{loss_alt_central}')
    res = pd.DataFrame(res,columns=['chrom','pos','ref','alt',
        f'{model_name}-l1',f'{model_name}-l2',f'{model_name}-dot',f'{model_name}-cosine',
        f'{model_name}-loss_ref_avg',f'{model_name}-loss_ref_cent', f'{model_name}-loss_alt_avg',f'{model_name}-loss_alt_cent'])
    return res

In [139]:
concat_embeddings('DNABERT-2')

loadding embeddings for DNABERT-2


100%|██████████| 2828/2828 [01:20<00:00, 35.03it/s]


putting all embeddings into a single file
deleting batch files
done


In [ ]:
model_name = 'DNABERT-2'

emb_file = embeddings_dir + f'{model_name}/embeddings.pickle'
with open(emb_file, 'rb') as f:
    seq_names,embeddings,losses = zip(*pickle.load(f))

compute_embeddings_score(seq_names,embeddings,losses,model_name)

In [14]:
utr_variants.to_csv(data_dir + '../perbase_pred/model_scores_snp.tsv', sep='\t', index=None)